# Using Stardog with Databricks from local Jupyter

Uninstall PySpark. This is required because the databricks-connect package conflicts with PySpark. For details, see Conflicting PySpark installations. Bash Copy. pip uninstall pyspark.
Install the Databricks Connect client. Bash Copy. pip install -U "databricks-connect==7.3. *" # or X.Y.

The required libraries (installed with pip or conda) include
1. `pyspark` (or `databricks-connect`)
1. `findspark` to locate the Spark installation and set initial config



In [ ]:
import findspark
findspark.init()

import pyspark
import random

import stardog
import io

import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/spark/user-jars/tg-jdbc-driver-1.2.jar pyspark-shell'


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
import pandas as pd
from configparser import ConfigParser


In [ ]:
spark = SparkSession.builder.appName("StardogAnalysis").getOrCreate()

In [ ]:
# define connection parameters
# file contains DEFAULT section plus override sections
config_section = 'doghouse'
parser = ConfigParser()
_ = parser.read('../CREDENTIALS.config')

url = parser.get(config_section, 'url')
user = parser.get(config_section, 'user')
password = parser.get(config_section, 'password')
db = 'insurance'
api_endpoint = 'query'

In [ ]:
connection_details = {
    'endpoint': url,
    'username': user,
    'password': password
}


In [ ]:
conn = stardog.Connection('insurance', **connection_details)

In [ ]:
query = """
PREFIX sqs: <tag:stardog:api:sqs:>
PREFIX : <http://api.stardog.com/>

select * { ?statIRI a :Crime_Stats;
                   :Crime_Type ?offense;
                   :Crime_Count ?crimeCount;
                   :Crime_Zip ?zipCode;
                   :Occurred_In ?zipCodeIri.
          ?zipCodeIri a :Zip_Codes.
          } 
"""

csv_results = conn.select(query, content_type='text/csv')
df = pd.read_csv(io.BytesIO(csv_results))

In [ ]:
df.head()

In [ ]:
spark.sql('show databases').show()

In [ ]:
jdbcDF = spark.read \
  .format("jdbc") \
  .option("driver", "com.tigergraph.jdbc.Driver") \
  .option("url", "jdbc:tg:" + gsurl) \
  .option("user", gsuser) \
  .option("password", gspass) \
  .option("graph", graph) \
  .option("dbtable", "vertex Orders") \
  .option("limit", "10") \
  .option("debug", "0") \
  .load()

jdbcDF.printSchema()
# jdbcDF.show()


In [ ]:
df = jdbcDF.toPandas()
df

In [ ]:
jdbcDF.show()

In [ ]:
!pip list